<a href="https://colab.research.google.com/github/erickgnfi-dotcom/ETL-restcountries-API/blob/main/ETL_restcountries_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
ETL Pipeline - CoinGecko
Arquitectura Medallion: Bronze → Silver → Gold
"""

import requests
import pandas as pd
import json
from datetime import datetime
from typing import Dict, List
import time

In [188]:
"""
Extrae datos crudos de la API y los almacena sin transofmar
"""
response = requests.get(f'https://restcountries.com/v3.1/all?fields=name,currencies,region,area,continents,languages,population')
response.json()[1]

{'name': {'common': 'Chile',
  'official': 'Republic of Chile',
  'nativeName': {'spa': {'official': 'República de Chile',
    'common': 'Chile'}}},
 'currencies': {'CLP': {'name': 'Chilean peso', 'symbol': '$'}},
 'region': 'Americas',
 'languages': {'spa': 'Spanish'},
 'area': 756102.0,
 'population': 19116209,
 'continents': ['South America']}

In [236]:
list(response.json()[1]['languages'].values())[0]

'Spanish'

In [238]:
"""Transforma y limpia los datos del bronze layer"""
df = []
for country in response.json():
    country_data = {'Pais': country['name']['common']}
    try:
        # Attempt to access the currency name
        country_data['Moneda'] = list(country['currencies'].values())[0]['name']
    except (KeyError, IndexError):
        # Handle cases where 'currencies' key is missing, dictionary is empty,
        # or index is out of range
        country_data['Moneda'] = None # Or some other placeholder like 'N/A'
    country_data['Region'] = country['region']
    country_data['Area'] = country['area']
    country_data['Continente'] = country['continents'][0]
    try:
      country_data['Idiomas'] = list(country['languages'].values())[0]
    except (KeyError, IndexError):
        country_data['Idiomas'] = None
    country_data['Poblacion'] = country['population']
    df.append(country_data)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(df)
df.head()

,Pais,Moneda,Region,Area,Continente,Idiomas,Poblacion
0,Lithuania,Euro,Europe,65300.0,Europe,Lithuanian,2794700
1,Chile,Chilean peso,Americas,756102.0,South America,Spanish,19116209
2,Benin,West African CFA franc,Africa,112622.0,Africa,French,12123198
3,Falkland Islands,Falkland Islands pound,Americas,12173.0,South America,English,2563
4,Georgia,lari,Asia,69700.0,Asia,Georgian,3714000
